In [2]:
import pandas as pd
import numpy as np
import spacy
from tqdm import tqdm
import re
import time
import pickle

In [3]:
df_headline = pd.read_csv('/home/ashish/FiQA/Task1/task1_headline_ABSA_train.csv')

In [4]:
df_headline.head()

,id,sentence,info_snippets,info_target,info_sentiment_score,info_aspects
0,1,Royal Mail chairman Donald Brydon set to step ...,['set to step down'],Royal Mail,-0.374,['Corporate/Appointment']
1,2,Stakes High for AstraZeneca Heart Drug Facing ...,['Facing Tough Competition'],AstraZeneca,-0.240,['Corporate/Risks']
2,3,UPDATE 1-Dairy Crest loses a third of Morrison...,['Crest loses a third of Morrisons milk contra...,Morrisons,-0.161,['Corporate/Sales/Failed Contract Discussion']
3,4,Insight hires Aviva's David Hillier for multi-...,['hires Aviva's David Hillier for multi-asset ...,Insight,0.137,['Corporate/Appointment/Executive Appointment']
4,5,Primark racks up a happy Christmas after stron...,['after strong sales'],Primark,0.704,['Corporate/Sales']


In [5]:
df_post = pd.read_csv('/home/ashish/FiQA/Task1/task1_post_ABSA_train.csv')

In [6]:
df_post.head()

,id,sentence,info_snippets,info_sentiment_score,info_target,info_aspects
0,14860,Slowly adding some $FIO here but gotta be care...,['Slowly adding some $FIO here but gotta be ca...,0.459,FIO,['Stock/Price Action/Bullish/Bull Position']
1,14864,$TRX http://stks.co/1KkK Long setup. MACD cross.,['Long setup. MACD cross.'],0.438,TRX,['Stock/Technical Analysis']
2,14867,I am not optimistic about $amzn both fundement...,['both fundementals and charts look like poopo...,-0.506,AMZN,['Stock/Price Action/Bearish']
3,14875,$GRPN might be selling off ahead of $P earning...,['might be selling off ahead'],-0.202,P,['Stock/Price Action/Bearish/Bearish Behavior']
4,14876,$IACI http://stks.co/tJU Looks good on the wee...,['Looks good on the weekly chart.'],0.379,IACI,['Stock/Technical Analysis']


In [7]:
df_headline.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 498 entries, 0 to 497
Data columns (total 6 columns):
id                      498 non-null int64
sentence                498 non-null object
info_snippets           498 non-null object
info_target             498 non-null object
info_sentiment_score    498 non-null float64
info_aspects            498 non-null object
dtypes: float64(1), int64(1), object(4)
memory usage: 23.4+ KB


In [8]:
df_headline.describe()

,id,info_sentiment_score
count,498.000000,498.000000
mean,249.500000,0.093129
std,143.904482,0.375713
min,1.000000,-0.938000
25%,125.250000,-0.184750
50%,249.500000,0.189000
75%,373.750000,0.364750
max,498.000000,0.975000


In [9]:
df_post.describe()

,id,info_sentiment_score
count,675.000000,675.000000
mean,16694.974815,0.143124
std,1237.448856,0.414214
min,14860.000000,-0.745000
25%,15622.500000,-0.301000
50%,16517.000000,0.318000
75%,17732.500000,0.476500
max,19167.000000,0.814000


In [10]:
df_headline.shape,df_post.shape

((498, 6), (675, 6))

In [11]:
df_headline['info_sentiment_score'].value_counts(normalize = True)

 0.000    0.026104
-0.158    0.006024
 0.276    0.006024
 0.214    0.006024
 0.295    0.006024
 0.345    0.006024
 0.329    0.006024
 0.199    0.006024
 0.224    0.006024
 0.365    0.006024
 0.284    0.006024
 0.281    0.006024
 0.259    0.006024
 0.066    0.006024
 0.250    0.006024
 0.143    0.006024
 0.144    0.004016
 0.037    0.004016
 0.571    0.004016
 0.172    0.004016
 0.461    0.004016
 0.213    0.004016
 0.516    0.004016
 0.437    0.004016
-0.604    0.004016
 0.439    0.004016
 0.189    0.004016
-0.398    0.004016
 0.257    0.004016
 0.408    0.004016
            ...   
-0.341    0.002008
 0.166    0.002008
-0.242    0.002008
-0.590    0.002008
-0.045    0.002008
 0.354    0.002008
 0.650    0.002008
 0.975    0.002008
 0.327    0.002008
 0.256    0.002008
 0.333    0.002008
-0.545    0.002008
 0.611    0.002008
-0.508    0.002008
 0.401    0.002008
-0.269    0.002008
-0.134    0.002008
-0.357    0.002008
 0.589    0.002008
 0.103    0.002008
 0.551    0.002008
-0.302    0.

In [12]:
# remove URL's from train and test
df_headline['sentence'] = df_headline['sentence'].apply(lambda x: re.sub(r'http\S+', '', x))
df_headline['info_snippets'] = df_headline['info_snippets'].apply(lambda x: re.sub(r'http\S+', '', x))

df_post['sentence'] = df_post['sentence'].apply(lambda x: re.sub(r'http\S+', '', x))
df_post['info_snippets'] = df_post['info_snippets'].apply(lambda x: re.sub(r'http\S+', '', x))

In [13]:
df_headline.head()

,id,sentence,info_snippets,info_target,info_sentiment_score,info_aspects
0,1,Royal Mail chairman Donald Brydon set to step ...,['set to step down'],Royal Mail,-0.374,['Corporate/Appointment']
1,2,Stakes High for AstraZeneca Heart Drug Facing ...,['Facing Tough Competition'],AstraZeneca,-0.240,['Corporate/Risks']
2,3,UPDATE 1-Dairy Crest loses a third of Morrison...,['Crest loses a third of Morrisons milk contra...,Morrisons,-0.161,['Corporate/Sales/Failed Contract Discussion']
3,4,Insight hires Aviva's David Hillier for multi-...,['hires Aviva's David Hillier for multi-asset ...,Insight,0.137,['Corporate/Appointment/Executive Appointment']
4,5,Primark racks up a happy Christmas after stron...,['after strong sales'],Primark,0.704,['Corporate/Sales']


In [14]:
# remove punctuation marks
punctuation = '!"#$%&()*+-/:;<=>?@[\\]^_.`{|}~'

df_headline['sentence'] = df_headline['sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))
df_post['sentence'] = df_post['sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))

df_headline['info_snippets'] = df_headline['info_snippets'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))
df_post['info_snippets'] = df_post['info_snippets'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))

# convert text to lowercase
df_headline['sentence'] = df_headline['sentence'].str.lower()
df_post['sentence'] = df_post['sentence'].str.lower()

df_headline['info_snippets'] = df_headline['info_snippets'].str.lower()
df_post['info_snippets'] = df_post['info_snippets'].str.lower()
# remove numbers
df_headline['sentence'] = df_headline['sentence'].str.replace("[0-9]", " ")
df_post['sentence'] = df_post['sentence'].str.replace("[0-9]", " ")

df_headline['info_snippets'] = df_headline['info_snippets'].str.replace("[0-9]", " ")
df_post['info_snippets'] = df_post['info_snippets'].str.replace("[0-9]", " ")
# remove whitespaces
df_headline['sentence'] = df_headline['sentence'].apply(lambda x:' '.join(x.split()))
df_post['sentence'] = df_post['sentence'].apply(lambda x: ' '.join(x.split()))

df_headline['info_snippets'] = df_headline['info_snippets'].apply(lambda x:' '.join(x.split()))
df_post['info_snippets'] = df_post['info_snippets'].apply(lambda x: ' '.join(x.split()))

In [15]:
# import spaCy's language model
nlp = spacy.load('en', disable=['parser', 'ner'])

# function to lemmatize text
def lemmatization(texts):
    output = []
    for i in texts:
        s = [token.lemma_ for token in nlp(i)]
        output.append(' '.join(s))
    return output

In [16]:
df_headline['sentence']  = lemmatization(df_headline['sentence'] )
df_post['sentence'] = lemmatization(df_post['sentence'] )

df_headline['info_snippets']  = lemmatization(df_headline['info_snippets'] )
df_post['info_snippets'] = lemmatization(df_post['info_snippets'] )

In [17]:
df_headline['info_sentiment_score'][df_headline['info_sentiment_score'] < 0] = 0

/home/ashish/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [18]:
df_headline['info_sentiment_score'][df_headline['info_sentiment_score'] > 0] = 1

/home/ashish/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [19]:
df_headline.head()

,id,sentence,info_snippets,info_target,info_sentiment_score,info_aspects
0,1,royal mail chairman donald brydon set to step ...,' set to step down ',Royal Mail,0.0,['Corporate/Appointment']
1,2,stake high for astrazeneca heart drug face tou...,' face tough competition ',AstraZeneca,0.0,['Corporate/Risks']
2,3,update dairy crest lose a third of morrison mi...,' crest lose a third of morrison milk contract ',Morrisons,0.0,['Corporate/Sales/Failed Contract Discussion']
3,4,insight hire aviva 's david hillier for multia...,' hire aviva 's david hillier for multiasset t...,Insight,1.0,['Corporate/Appointment/Executive Appointment']
4,5,primark rack up a happy christma after strong ...,' after strong sale ',Primark,1.0,['Corporate/Sales']


In [20]:
df_headline.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 498 entries, 0 to 497
Data columns (total 6 columns):
id                      498 non-null int64
sentence                498 non-null object
info_snippets           498 non-null object
info_target             498 non-null object
info_sentiment_score    498 non-null float64
info_aspects            498 non-null object
dtypes: float64(1), int64(1), object(4)
memory usage: 23.4+ KB


In [21]:
df_headline[['info_sentiment_score']] = df_headline[['info_sentiment_score']].apply(pd.to_numeric) 

In [22]:
# using dictionary to convert specific columns 
convert_dict = {'info_sentiment_score': int
               } 
  
df_headline = df_headline.astype(convert_dict) 
print(df_headline.dtypes)

id                       int64
sentence                object
info_snippets           object
info_target             object
info_sentiment_score     int64
info_aspects            object
dtype: object


In [23]:
df_headline.head()

,id,sentence,info_snippets,info_target,info_sentiment_score,info_aspects
0,1,royal mail chairman donald brydon set to step ...,' set to step down ',Royal Mail,0,['Corporate/Appointment']
1,2,stake high for astrazeneca heart drug face tou...,' face tough competition ',AstraZeneca,0,['Corporate/Risks']
2,3,update dairy crest lose a third of morrison mi...,' crest lose a third of morrison milk contract ',Morrisons,0,['Corporate/Sales/Failed Contract Discussion']
3,4,insight hire aviva 's david hillier for multia...,' hire aviva 's david hillier for multiasset t...,Insight,1,['Corporate/Appointment/Executive Appointment']
4,5,primark rack up a happy christma after strong ...,' after strong sale ',Primark,1,['Corporate/Sales']


In [24]:
# Import label encoder 
from sklearn import preprocessing 
  
# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in column 'species'. 
df_headline['info_target']= label_encoder.fit_transform(df_headline['info_target']) 
  
df_headline['info_target'].unique()

array([170,  14, 141, 104, 160, 153, 197, 199, 210, 169, 185,  47, 201,
        68,  33, 149, 151, 202,  99,  95, 155,  24, 165, 138, 215,  36,
       130, 147,  15,   1,  54,  21, 132, 181, 207, 191, 192,  65, 139,
       163,  90, 122,  59,  72,  17, 168, 217,  31,  77,  76,  16,  56,
         8, 200, 176, 144,  13,  27, 195, 152, 188, 206, 162, 161,  41,
       119,  57, 143, 222, 184, 136, 137, 164, 205,  75, 190, 196,  62,
        82, 224, 214, 211,  85,  84, 102, 174, 187,  69, 150,  91,  20,
       158,  92,  45, 182, 105, 123,  28, 140, 117, 129,  81, 194,   5,
       183,  42,  10, 121, 223, 189, 186, 127, 225, 114, 157,   7, 128,
       204, 193,  94,  35, 221, 146,  55, 209, 179,  19,  83, 166, 135,
        79,  70,  86,  38,   9, 103,  22,  40,   4, 120,  32,  12, 213,
        37,  48,  93,  98, 142,   6,   2, 100,  49, 173,  80, 145,  23,
       133,  39, 116,  73, 177, 220, 212, 154,  53,  64,  34, 115, 218,
        52, 101,  97,  58, 208,  96, 178,  29,  18, 175,  89, 10

In [25]:
df_headline.head()

,id,sentence,info_snippets,info_target,info_sentiment_score,info_aspects
0,1,royal mail chairman donald brydon set to step ...,' set to step down ',170,0,['Corporate/Appointment']
1,2,stake high for astrazeneca heart drug face tou...,' face tough competition ',14,0,['Corporate/Risks']
2,3,update dairy crest lose a third of morrison mi...,' crest lose a third of morrison milk contract ',141,0,['Corporate/Sales/Failed Contract Discussion']
3,4,insight hire aviva 's david hillier for multia...,' hire aviva 's david hillier for multiasset t...,104,1,['Corporate/Appointment/Executive Appointment']
4,5,primark rack up a happy christma after strong ...,' after strong sale ',160,1,['Corporate/Sales']


In [26]:
import os
import collections
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

In [27]:
!pip install bert-tensorflow

In [28]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 829757256083421493
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 17073840334390304174
physical_device_desc: "device: XLA_CPU device"
]


In [29]:
tf.test.is_gpu_available(
    cuda_only=False,
    min_cuda_compute_capability=None
)

False

In [30]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
from bert import modeling

In [31]:
##use downloaded model, change path accordingly
BERT_VOCAB= '/home/ashish/FiQA/uncased_L-12_H-768_A-12/vocab.txt'
BERT_INIT_CHKPNT = '/home/ashish/FiQA/uncased_L-12_H-768_A-12/bert_model.ckpt'
BERT_CONFIG = '/home/ashish/FiQA/uncased_L-12_H-768_A-12/bert_config.json'

In [32]:
tokenization.validate_case_matches_checkpoint(True,BERT_INIT_CHKPNT)
tokenizer = tokenization.FullTokenizer(
      vocab_file=BERT_VOCAB, do_lower_case=True)

In [33]:
tokenizer.tokenize("This here's an example of using the BERT tokenizer")

['this',
 'here',
 "'",
 's',
 'an',
 'example',
 'of',
 'using',
 'the',
 'bert',
 'token',
 '##izer']

In [34]:
ID = 'id'
DATA_COLUMN = 'sentence'
LABEL_COLUMNS = ['info_target','info_sentiment_score']

In [35]:
class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, labels=None):
        """Constructs a InputExample.

        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
            labels: (Optional) [string]. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.labels = labels


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_ids, is_real_example=True):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_ids = label_ids,
        self.is_real_example=is_real_example

In [36]:
df_headline.iloc[:,3:-1]

,info_target,info_sentiment_score
0,170,0
1,14,0
2,141,0
3,104,1
4,160,1
5,153,1
6,197,1
7,199,1
8,210,1
9,199,1


In [37]:
def create_examples(df, labels_available=True):
    """Creates examples for the training and dev sets."""
    examples = []
    for (i, row) in enumerate(df.values):
        guid = row[0]
        text_a = row[1]
        if labels_available:
            labels = row[3:-1]
        else:
            labels = [0,0]
        examples.append(
            InputExample(guid=guid, text_a=text_a, labels=labels))
    return examples

In [38]:
TRAIN_VAL_RATIO = 0.7
LEN = df_headline.shape[0]
SIZE_TRAIN = int(TRAIN_VAL_RATIO*LEN)

x_train = df_headline[:SIZE_TRAIN]
x_val = df_headline[SIZE_TRAIN:]

train_examples = create_examples(x_train)

In [39]:
def convert_examples_to_features(examples,  max_seq_length, tokenizer):
    """Loads a data file into a list of `InputBatch`s."""

    features = []
    for (ex_index, example) in enumerate(examples):
        print(example.text_a)
        tokens_a = tokenizer.tokenize(example.text_a)

        tokens_b = None
        if example.text_b:
            tokens_b = tokenizer.tokenize(example.text_b)
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > max_seq_length - 2:
                tokens_a = tokens_a[:(max_seq_length - 2)]

        # The convention in BERT is:
        # (a) For sequence pairs:
        #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
        #  type_ids: 0   0  0    0    0     0       0 0    1  1  1  1   1 1
        # (b) For single sequences:
        #  tokens:   [CLS] the dog is hairy . [SEP]
        #  type_ids: 0   0   0   0  0     0 0
        #
        # Where "type_ids" are used to indicate whether this is the first
        # sequence or the second sequence. The embedding vectors for `type=0` and
        # `type=1` were learned during pre-training and are added to the wordpiece
        # embedding vector (and position vector). This is not *strictly* necessary
        # since the [SEP] token unambigiously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.
        #
        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens += tokens_b + ["[SEP]"]
            segment_ids += [1] * (len(tokens_b) + 1)

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1] * len(input_ids)

        # Zero-pad up to the sequence length.
        padding = [0] * (max_seq_length - len(input_ids))
        input_ids += padding
        input_mask += padding
        segment_ids += padding

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length
        
        labels_ids = []
        for label in example.labels:
            labels_ids.append(int(label))

        if ex_index < 0:
            logger.info("*** Example ***")
            logger.info("guid: %s" % (example.guid))
            logger.info("tokens: %s" % " ".join(
                    [str(x) for x in tokens]))
            logger.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
            logger.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
            logger.info(
                    "segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
            logger.info("label: %s (id = %s)" % (example.labels, labels_ids))

        features.append(
                InputFeatures(input_ids=input_ids,
                              input_mask=input_mask,
                              segment_ids=segment_ids,
                              label_ids=labels_ids))
    return features

In [40]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128

In [41]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 8
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 1.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 1000
SAVE_SUMMARY_STEPS = 500

In [42]:
class PaddingInputExample(object):
    """Fake example so the num input examples is a multiple of the batch size.
    When running eval/predict on the TPU, we need to pad the number of examples
    to be a multiple of the batch size, because the TPU requires a fixed batch
    size. The alternative is to drop the last batch, which is bad because it means
    the entire output data won't be generated.
    We use this class instead of `None` because treating `None` as padding
    battches could cause silent errors.
    """
    
    
def convert_single_example(ex_index, example, max_seq_length,
                           tokenizer):
    """Converts a single `InputExample` into a single `InputFeatures`."""

    if isinstance(example, PaddingInputExample):
        return InputFeatures(
            input_ids=[0] * max_seq_length,
            input_mask=[0] * max_seq_length,
            segment_ids=[0] * max_seq_length,
            label_ids=0,
            is_real_example=False)

    tokens_a = tokenizer.tokenize(example.text_a)
    tokens_b = None
    if example.text_b:
        tokens_b = tokenizer.tokenize(example.text_b)

    if tokens_b:
        # Modifies `tokens_a` and `tokens_b` in place so that the total
        # length is less than the specified length.
        # Account for [CLS], [SEP], [SEP] with "- 3"
        _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
    else:
        # Account for [CLS] and [SEP] with "- 2"
        if len(tokens_a) > max_seq_length - 2:
            tokens_a = tokens_a[0:(max_seq_length - 2)]

    # The convention in BERT is:
    # (a) For sequence pairs:
    #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
    #  type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
    # (b) For single sequences:
    #  tokens:   [CLS] the dog is hairy . [SEP]
    #  type_ids: 0     0   0   0  0     0 0
    #
    # Where "type_ids" are used to indicate whether this is the first
    # sequence or the second sequence. The embedding vectors for `type=0` and
    # `type=1` were learned during pre-training and are added to the wordpiece
    # embedding vector (and position vector). This is not *strictly* necessary
    # since the [SEP] token unambiguously separates the sequences, but it makes
    # it easier for the model to learn the concept of sequences.
    #
    # For classification tasks, the first vector (corresponding to [CLS]) is
    # used as the "sentence vector". Note that this only makes sense because
    # the entire model is fine-tuned.
    tokens = []
    segment_ids = []
    tokens.append("[CLS]")
    segment_ids.append(0)
    for token in tokens_a:
        tokens.append(token)
        segment_ids.append(0)
    tokens.append("[SEP]")
    segment_ids.append(0)

    if tokens_b:
        for token in tokens_b:
            tokens.append(token)
            segment_ids.append(1)
        tokens.append("[SEP]")
        segment_ids.append(1)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1] * len(input_ids)

    # Zero-pad up to the sequence length.
    while len(input_ids) < max_seq_length:
        input_ids.append(0)
        input_mask.append(0)
        segment_ids.append(0)

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length

    labels_ids = []
    for label in example.labels:
        labels_ids.append(int(label))


    feature = InputFeatures(
        input_ids=input_ids,
        input_mask=input_mask,
        segment_ids=segment_ids,
        label_ids=labels_ids,
        is_real_example=True)
    return feature


def file_based_convert_examples_to_features(
        examples, max_seq_length, tokenizer, output_file):
    """Convert a set of `InputExample`s to a TFRecord file."""

    writer = tf.python_io.TFRecordWriter(output_file)

    for (ex_index, example) in enumerate(examples):
        #if ex_index % 10000 == 0:
            #tf.logging.info("Writing example %d of %d" % (ex_index, len(examples)))

        feature = convert_single_example(ex_index, example,
                                         max_seq_length, tokenizer)

        def create_int_feature(values):
            f = tf.train.Feature(int64_list=tf.train.Int64List(value=list(values)))
            return f

        features = collections.OrderedDict()
        features["input_ids"] = create_int_feature(feature.input_ids)
        features["input_mask"] = create_int_feature(feature.input_mask)
        features["segment_ids"] = create_int_feature(feature.segment_ids)
        features["is_real_example"] = create_int_feature(
            [int(feature.is_real_example)])
        if isinstance(feature.label_ids, list):
            label_ids = feature.label_ids
        else:
            label_ids = feature.label_ids[0]
        features["label_ids"] = create_int_feature(label_ids)

        tf_example = tf.train.Example(features=tf.train.Features(feature=features))
        writer.write(tf_example.SerializeToString())
    writer.close()


def file_based_input_fn_builder(input_file, seq_length, is_training,
                                drop_remainder):
    """Creates an `input_fn` closure to be passed to TPUEstimator."""

    name_to_features = {
        "input_ids": tf.FixedLenFeature([seq_length], tf.int64),
        "input_mask": tf.FixedLenFeature([seq_length], tf.int64),
        "segment_ids": tf.FixedLenFeature([seq_length], tf.int64),
        "label_ids": tf.FixedLenFeature([2], tf.int64),
        "is_real_example": tf.FixedLenFeature([], tf.int64),
    }

    def _decode_record(record, name_to_features):
        """Decodes a record to a TensorFlow example."""
        example = tf.parse_single_example(record, name_to_features)

        # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
        # So cast all int64 to int32.
        for name in list(example.keys()):
            t = example[name]
            if t.dtype == tf.int64:
                t = tf.to_int32(t)
            example[name] = t

        return example

    def input_fn(params):
        """The actual input function."""
        batch_size = params["batch_size"]

        # For training, we want a lot of parallel reading and shuffling.
        # For eval, we want no shuffling and parallel reading doesn't matter.
        d = tf.data.TFRecordDataset(input_file)
        if is_training:
            d = d.repeat()
            d = d.shuffle(buffer_size=100)

        d = d.apply(
            tf.contrib.data.map_and_batch(
                lambda record: _decode_record(record, name_to_features),
                batch_size=batch_size,
                drop_remainder=drop_remainder))

        return d

    return input_fn


def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()

In [43]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_examples) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [44]:
num_train_steps

43

In [45]:
train_file = os.path.join('Bert/working', "train.tf_record")
#filename = Path(train_file)
if not os.path.exists(train_file):
    open(train_file, 'w').close()

In [46]:
file_based_convert_examples_to_features(
            train_examples, MAX_SEQ_LENGTH, tokenizer, train_file)
tf.logging.info("***** Running training *****")
tf.logging.info("  Num examples = %d", len(train_examples))
tf.logging.info("  Batch size = %d", BATCH_SIZE)
tf.logging.info("  Num steps = %d", num_train_steps)

INFO:tensorflow:***** Running training *****
INFO:tensorflow:  Num examples = 348
INFO:tensorflow:  Batch size = 8
INFO:tensorflow:  Num steps = 43


In [47]:
train_input_fn = file_based_input_fn_builder(
    input_file=train_file,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=True)

In [48]:
def create_model(bert_config, is_training, input_ids, input_mask, segment_ids,
                 labels, num_labels, use_one_hot_embeddings):
    """Creates a classification model."""
    model = modeling.BertModel(
        config=bert_config,
        is_training=is_training,
        input_ids=input_ids,
        input_mask=input_mask,
        token_type_ids=segment_ids,
        use_one_hot_embeddings=use_one_hot_embeddings)

    # In the demo, we are doing a simple classification task on the entire
    # segment.
    #
    # If you want to use the token-level output, use model.get_sequence_output()
    # instead.
    output_layer = model.get_pooled_output()

    hidden_size = output_layer.shape[-1].value

    output_weights = tf.get_variable(
        "output_weights", [num_labels, hidden_size],
        initializer=tf.truncated_normal_initializer(stddev=0.02))

    output_bias = tf.get_variable(
        "output_bias", [num_labels], initializer=tf.zeros_initializer())

    with tf.variable_scope("loss"):
        if is_training:
            # I.e., 0.1 dropout
            output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

        logits = tf.matmul(output_layer, output_weights, transpose_b=True)
        logits = tf.nn.bias_add(logits, output_bias)
        
        # probabilities = tf.nn.softmax(logits, axis=-1) ### multiclass case
        probabilities = tf.nn.sigmoid(logits)#### multi-label case
        
        labels = tf.cast(labels, tf.float32)
        tf.logging.info("num_labels:{};logits:{};labels:{}".format(num_labels, logits, labels))
        per_example_loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=labels, logits=logits)
        loss = tf.reduce_mean(per_example_loss)

        # probabilities = tf.nn.softmax(logits, axis=-1)
        # log_probs = tf.nn.log_softmax(logits, axis=-1)
        #
        # one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)
        #
        # per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
        # loss = tf.reduce_mean(per_example_loss)

        return (loss, per_example_loss, logits, probabilities)


def model_fn_builder(bert_config, num_labels, init_checkpoint, learning_rate,
                     num_train_steps, num_warmup_steps, use_tpu,
                     use_one_hot_embeddings):
    """Returns `model_fn` closure for TPUEstimator."""

    def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
        """The `model_fn` for TPUEstimator."""

        #tf.logging.info("*** Features ***")
        #for name in sorted(features.keys()):
        #    tf.logging.info("  name = %s, shape = %s" % (name, features[name].shape))

        input_ids = features["input_ids"]
        input_mask = features["input_mask"]
        segment_ids = features["segment_ids"]
        label_ids = features["label_ids"]
        is_real_example = None
        if "is_real_example" in features:
             is_real_example = tf.cast(features["is_real_example"], dtype=tf.float32)
        else:
             is_real_example = tf.ones(tf.shape(label_ids), dtype=tf.float32)

        is_training = (mode == tf.estimator.ModeKeys.TRAIN)

        (total_loss, per_example_loss, logits, probabilities) = create_model(
            bert_config, is_training, input_ids, input_mask, segment_ids, label_ids,
            num_labels, use_one_hot_embeddings)

        tvars = tf.trainable_variables()
        initialized_variable_names = {}
        scaffold_fn = None
        if init_checkpoint:
            (assignment_map, initialized_variable_names
             ) = modeling.get_assignment_map_from_checkpoint(tvars, init_checkpoint)
            if use_tpu:

                def tpu_scaffold():
                    tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
                    return tf.train.Scaffold()

                scaffold_fn = tpu_scaffold
            else:
                tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

        tf.logging.info("**** Trainable Variables ****")
        for var in tvars:
            init_string = ""
            if var.name in initialized_variable_names:
                init_string = ", *INIT_FROM_CKPT*"
            #tf.logging.info("  name = %s, shape = %s%s", var.name, var.shape,init_string)

        output_spec = None
        if mode == tf.estimator.ModeKeys.TRAIN:

            train_op = optimization.create_optimizer(
                total_loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu)

            output_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                loss=total_loss,
                train_op=train_op,
                scaffold=scaffold_fn)
        elif mode == tf.estimator.ModeKeys.EVAL:

            def metric_fn(per_example_loss, label_ids, probabilities, is_real_example):

                logits_split = tf.split(probabilities, num_labels, axis=-1)
                label_ids_split = tf.split(label_ids, num_labels, axis=-1)
                # metrics change to auc of every class
                eval_dict = {}
                for j, logits in enumerate(logits_split):
                    label_id_ = tf.cast(label_ids_split[j], dtype=tf.int32)
                    current_auc, update_op_auc = tf.metrics.auc(label_id_, logits)
                    eval_dict[str(j)] = (current_auc, update_op_auc)
                eval_dict['eval_loss'] = tf.metrics.mean(values=per_example_loss)
                return eval_dict

                ## original eval metrics
                # predictions = tf.argmax(logits, axis=-1, output_type=tf.int32)
                # accuracy = tf.metrics.accuracy(
                #     labels=label_ids, predictions=predictions, weights=is_real_example)
                # loss = tf.metrics.mean(values=per_example_loss, weights=is_real_example)
                # return {
                #     "eval_accuracy": accuracy,
                #     "eval_loss": loss,
                # }

            eval_metrics = metric_fn(per_example_loss, label_ids, probabilities, is_real_example)
            output_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                loss=total_loss,
                eval_metric_ops=eval_metrics,
                scaffold=scaffold_fn)
        else:
            print("mode:", mode,"probabilities:", probabilities)
            output_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                predictions={"probabilities": probabilities},
                scaffold=scaffold_fn)
        return output_spec

    return model_fn

In [49]:
OUTPUT_DIR = "Bert/working/output"
# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    keep_checkpoint_max=1,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [50]:
bert_config = modeling.BertConfig.from_json_file(BERT_CONFIG)
model_fn = model_fn_builder(
  bert_config=bert_config,
  num_labels= len(LABEL_COLUMNS),
  init_checkpoint=BERT_INIT_CHKPNT,
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps,
  use_tpu=False,
  use_one_hot_embeddings=False)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

INFO:tensorflow:Using config: {'_model_dir': 'Bert/working/output', '_tf_random_seed': None, '_save_summary_steps': 500, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fcbf82a8400>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [51]:
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use `tf.data.experimental.map_and_batch(...)`.
Instructions for updating:
Use `tf.data.Dataset.map(map_func, num_parallel_calls)` followed by `tf.data.Dataset.batch(batch_size, drop_remainder)`. Static tf.data optimizations will take care of using the fused implementation.
Instructions for updating:
Use `tf.cast` instead.
INFO:tensorflow:Calling model_fn.



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate sho

In [53]:
eval_file = os.path.join('Bert/working', "eval.tf_record")
#filename = Path(train_file)
if not os.path.exists(eval_file):
    open(eval_file, 'w').close()

eval_examples = create_examples(x_val)
file_based_convert_examples_to_features(
    eval_examples, MAX_SEQ_LENGTH, tokenizer, eval_file)

In [54]:
# This tells the estimator to run through the entire set.
eval_steps = None

eval_drop_remainder = False
eval_input_fn = file_based_input_fn_builder(
    input_file=eval_file,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:num_labels:2;logits:Tensor("loss/BiasAdd:0", shape=(?, 2), dtype=float32);labels:Tensor("loss/Cast:0", shape=(?, 2), dtype=float32)
INFO:tensorflow:**** Trainable Variables ****
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-11-06T11:49:21Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Bert/working/output/model.ckpt-43
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-11-06-11:49:54
INFO:tensorflow:Saving dict for global step 43: 0 = 0.49999952, 1 = 0.5188383, eval_loss = -424.4017, global_step = 43, loss = -425.34995
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 43: Bert/working/output/model.ckpt-43


In [55]:
output_eval_file = os.path.join("Bert/working", "eval_results.txt")
with tf.gfile.GFile(output_eval_file, "w") as writer:
    tf.logging.info("***** Eval results *****")
    for key in sorted(result.keys()):
        tf.logging.info("  %s = %s", key, str(result[key]))
        writer.write("%s = %s\n" % (key, str(result[key])))

INFO:tensorflow:***** Eval results *****
INFO:tensorflow:  0 = 0.49999952
INFO:tensorflow:  1 = 0.5188383
INFO:tensorflow:  eval_loss = -424.4017
INFO:tensorflow:  global_step = 43
INFO:tensorflow:  loss = -425.34995


In [67]:
df_headline_test = pd.read_csv('Task1/task1_headline_ABSA_test.csv')

In [68]:
df_post_test = pd.read_csv('Task1/task1_post_ABSA_test.csv')

In [69]:
df_headline_test.shape, df_post_test.shape

((93, 4), (99, 4))

In [70]:
x_test = df_headline_test
x_test = x_test.reset_index(drop=True)

In [71]:
x_test.head()

,id,sentence,info_snippets,info_target
0,0_Cuadrilla,Cuadrilla files to delay application to frack ...,['files to delay application'],Cuadrilla
1,1001_Sainsbury,Sainsbury chief warns of squeeze on high stree...,['warns of squeeze on high street retailers'],Sainsbury
2,1006_Barclays,Barclays fined for anti-money-laundering failings,['fined for anti-money-laundering failings'],Barclays
3,1007_Barclays,UPDATE 3-Barclays fined for lax crime checks i...,['fined for lax crime checks in'],Barclays
4,1014_GSK,GSK aims to file up to 20 new drugs for approv...,['file up to 20 new drugs for approval by 2020'],GSK


In [73]:
x_test_new = x_test[['id',	'sentence',	'info_snippets',	'info_target']]

In [74]:
predict_examples = create_examples(x_test_new,False)

In [75]:
x_test_new.head()

,id,sentence,info_snippets,info_target
0,0_Cuadrilla,Cuadrilla files to delay application to frack ...,['files to delay application'],Cuadrilla
1,1001_Sainsbury,Sainsbury chief warns of squeeze on high stree...,['warns of squeeze on high street retailers'],Sainsbury
2,1006_Barclays,Barclays fined for anti-money-laundering failings,['fined for anti-money-laundering failings'],Barclays
3,1007_Barclays,UPDATE 3-Barclays fined for lax crime checks i...,['fined for lax crime checks in'],Barclays
4,1014_GSK,GSK aims to file up to 20 new drugs for approv...,['file up to 20 new drugs for approval by 2020'],GSK


In [76]:
test_features = convert_examples_to_features(predict_examples, MAX_SEQ_LENGTH, tokenizer)

Cuadrilla files to delay application to frack in Lancashire
Sainsbury chief warns of squeeze on high street retailers
Barclays fined for anti-money-laundering failings
UPDATE 3-Barclays fined for lax crime checks in 'deal of century'
GSK aims to file up to 20 new drugs for approval by 2020
CompaniesNational Grid lines up sale of gas business
British American Tobacco accused of bribing senior politicians to sabotage ...
UPDATE 2-AB InBev launches SAB bid, to sell MillerCoors stake
AstraZeneca shares climb 3% as drug maker ups profits forecasts
Landlord Hammerson's NAV rises on increased leasing activity
BAE Systems cuts earnings target for 2015 and 370 jobs
Barclays settles with US investors over Libor manipulation
Bunzl backs 2015 view, buys more businesses
L&G strikes Â£230m bulk annuity deal
Bloomberg buys Barclays' benchmarking business
Barclays in $13.75 million US settlement over mutual funds
Tesco UK personnel director quits supermarket
Companies Severn Trent expects costs hit fr

In [77]:
def input_fn_builder(features, seq_length, is_training, drop_remainder):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""

  all_input_ids = []
  all_input_mask = []
  all_segment_ids = []
  all_label_ids = []

  for feature in features:
    all_input_ids.append(feature.input_ids)
    all_input_mask.append(feature.input_mask)
    all_segment_ids.append(feature.segment_ids)
    all_label_ids.append(feature.label_ids)

  def input_fn(params):
    """The actual input function."""
    batch_size = params["batch_size"]

    num_examples = len(features)

    # This is for demo purposes and does NOT scale to large data sets. We do
    # not use Dataset.from_generator() because that uses tf.py_func which is
    # not TPU compatible. The right way to load data is with TFRecordReader.
    d = tf.data.Dataset.from_tensor_slices({
        "input_ids":
            tf.constant(
                all_input_ids, shape=[num_examples, seq_length],
                dtype=tf.int32),
        "input_mask":
            tf.constant(
                all_input_mask,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "segment_ids":
            tf.constant(
                all_segment_ids,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "label_ids":
            tf.constant(all_label_ids, shape=[num_examples, len(LABEL_COLUMNS)], dtype=tf.int32),
    })

    if is_training:
      d = d.repeat()
      d = d.shuffle(buffer_size=100)

    d = d.batch(batch_size=batch_size, drop_remainder=drop_remainder)
    return d

  return input_fn

In [82]:
print('Beginning Predictions!')
current_time = datetime.now()

predict_input_fn = input_fn_builder(features=test_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
predictions = estimator.predict(predict_input_fn)
print("Prediction took time ", datetime.now() - current_time)

Beginning Predictions!
Prediction took time  0:00:00.000137


In [83]:
def create_output(predictions):
    probabilities = []
    for (i, prediction) in enumerate(predictions):
        preds = prediction["probabilities"]
        probabilities.append(preds)
    dff = pd.DataFrame(probabilities)
    dff.columns = LABEL_COLUMNS
    
    return dff

In [84]:
output_df = create_output(predictions)
merged_df =  pd.concat([x_test, output_df], axis=1)
results = merged_df.drop(['sentence'], axis=1)
results.to_csv("result.csv", index=False)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:num_labels:2;logits:Tensor("loss/BiasAdd:0", shape=(?, 2), dtype=float32);labels:Tensor("loss/Cast:0", shape=(?, 2), dtype=float32)
INFO:tensorflow:**** Trainable Variables ****
mode: infer probabilities: Tensor("loss/Sigmoid:0", shape=(?, 2), dtype=float32)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Bert/working/output/model.ckpt-43
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [85]:
results.head()

,id,info_snippets,info_target,info_target,info_sentiment_score
0,0_Cuadrilla,['files to delay application'],Cuadrilla,0.999147,0.604033
1,1001_Sainsbury,['warns of squeeze on high street retailers'],Sainsbury,0.999159,0.602696
2,1006_Barclays,['fined for anti-money-laundering failings'],Barclays,0.999122,0.609739
3,1007_Barclays,['fined for lax crime checks in'],Barclays,0.999141,0.607174
4,1014_GSK,['file up to 20 new drugs for approval by 2020'],GSK,0.999117,0.605187
